## Evaluate on CVRPLib

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# If you did not download the VRPLIB, you may run the following
from routefinder.data.download_vrplib import download_vrplib

download_vrplib() 

In [3]:
import torch

from routefinder.models.model import RouteFinderBase, RouteFinderMoE
from routefinder.models.baselines.mvmoe.model import MVMoE
from routefinder.models.baselines.mtpomo.model import MTPOMO
from routefinder.envs.mtvrp import MTVRPEnv, MTVRPGenerator


# Choose your model

PATH = "../checkpoints/100/rf-transformer.ckpt"
model = RouteFinderBase.load_from_checkpoint(PATH, map_location="cpu")

# PATH = "../checkpoints/100/mtpomo.ckpt"
# model = MTPOMO.load_from_checkpoint(PATH, map_location="cpu")

# PATH = "../checkpoints/100/mvmoe.ckpt"
# model = MVMoE.load_from_checkpoint(PATH, map_location="cpu")

/home/botu/mambaforge/envs/routefinder/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/botu/mambaforge/envs/routefinder/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'env' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['env'])`.
/home/botu/mambaforge/envs/routefinder/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.
Provided file name ['data/cvrp/val/100.npz', 'data/ovrp/val/100.npz', 'data/ovrpb/val/100.npz', 'data/ovrpbl/val/100.npz', 'data/ovrpbltw/val/100.npz',

In [4]:
policy = model.policy

# Create env
generator = MTVRPGenerator(num_loc=100, variant_preset="all")
env = MTVRPEnv(generator, check_solution=False)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
policy = policy.to(device).eval()

td_test = env.reset(env.generator(32))

# Test the model
with torch.amp.autocast("cuda"):
    with torch.inference_mode():
        out = policy(td_test.clone().to(device), env, phase="test", decode_type="greedy", return_actions=True)
        actions = out['actions'].cpu().detach()
        rewards = out['reward'].cpu().detach()

print(f"Average reward: {rewards.mean().item():.3f}")

Average reward: -19.634


### VRPLib Evaluation

Note: run 

```
python routefinder/data/download_vrplib.py
```
 to download the VRPLib instances (then move them under this dir)

In [ ]:
import os

# Ensure you have downloaded vrplib under vrplib/
# Initialize the instances dictionary
instances = {}

# Walk through the vrplib directory recursively
for root, dirs, files in sorted(os.walk('./vrplib')):
    for file in files:
        if file.endswith('.vrp'):
            # Initialize the dictionary for this instance
            instance_name = file[:-4]  # Remove the '.vrp' extension
            instances[instance_name] = {"solution": None}  # Create entry for instance
            # Print the file for verification
            instances[instance_name]["data"] = os.path.join(root, file)  # Save the VRP file path
            instances[instance_name]["solution"] = os.path.join(root, file[:-4] + '.sol')  # Save the solution file path
            # ensure the solution file exists
            assert os.path.exists(instances[instance_name]["solution"]), f"Solution file not found for {instance_name}"
            print(f"Found VRP file: {file}")

Found VRP file: A-n37-k6.vrp
Found VRP file: A-n33-k5.vrp
Found VRP file: A-n44-k6.vrp
Found VRP file: A-n48-k7.vrp
Found VRP file: A-n39-k5.vrp
Found VRP file: A-n63-k10.vrp
Found VRP file: A-n36-k5.vrp
Found VRP file: A-n34-k5.vrp
Found VRP file: A-n55-k9.vrp
Found VRP file: A-n69-k9.vrp
Found VRP file: A-n61-k9.vrp
Found VRP file: A-n38-k5.vrp
Found VRP file: A-n53-k7.vrp
Found VRP file: A-n45-k6.vrp
Found VRP file: A-n80-k10.vrp
Found VRP file: A-n46-k7.vrp
Found VRP file: A-n33-k6.vrp
Found VRP file: A-n37-k5.vrp
Found VRP file: A-n60-k9.vrp
Found VRP file: A-n39-k6.vrp
Found VRP file: A-n32-k5.vrp
Found VRP file: A-n45-k7.vrp
Found VRP file: A-n65-k9.vrp
Found VRP file: A-n64-k9.vrp
Found VRP file: A-n62-k8.vrp
Found VRP file: A-n63-k9.vrp
Found VRP file: A-n54-k7.vrp
Found VRP file: B-n41-k6.vrp
Found VRP file: B-n35-k5.vrp
Found VRP file: B-n38-k6.vrp
Found VRP file: B-n64-k9.vrp
Found VRP file: B-n39-k5.vrp
Found VRP file: B-n50-k8.vrp
Found VRP file: B-n45-k6.vrp
Found VRP fi

In [7]:
from rl4co.utils.ops import unbatchify, gather_by_index

# Utils function
def normalize_coord(coord:torch.Tensor) -> torch.Tensor: # if we scale x and y separately, aren't we losing the relative position of the points? i.e. we mess with the distances.
    x, y = coord[:, 0], coord[:, 1]
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    
    x_scaled = (x - x_min) / (x_max - x_min) 
    y_scaled = (y - y_min) / (y_max - y_min)
    coord_scaled = torch.stack([x_scaled, y_scaled], dim=1)
    return coord_scaled 


def evaluate(model, td, env,
             num_augment=8,
             num_starts=None,
             ):
    
    with torch.inference_mode():
        with torch.amp.autocast("cuda"):
            n_start = env.get_num_starts(td) if num_starts is None else num_starts

            if num_augment > 1:
                td = model.augment(td)

            # Evaluate policy
            out = model.policy(
                td, env, phase="test", num_starts=n_start, return_actions=True
            )

            # Unbatchify reward to [batch_size, num_augment, num_starts].
            reward = unbatchify(out["reward"], (num_augment, n_start))

            if n_start > 1:
                # max multi-start reward
                max_reward, max_idxs = reward.max(dim=-1)
                out.update({"max_reward": max_reward})

                if out.get("actions", None) is not None:
                    # Reshape batch to [batch_size, num_augment, num_starts, ...]
                    actions = unbatchify(out["actions"], (num_augment, n_start))
                    out.update(
                        {"best_multistart_actions": gather_by_index(actions, max_idxs, dim=max_idxs.dim())}
                    )
                    out["actions"] = actions

            # Get augmentation score only during inference
            if num_augment > 1:
                # If multistart is enabled, we use the best multistart rewards
                reward_ = max_reward if n_start > 1 else reward
                max_aug_reward, max_idxs = reward_.max(dim=1)
                out.update({"max_aug_reward": max_aug_reward})

                if out.get("actions", None) is not None:
                    actions_ = (
                        out["best_multistart_actions"] if n_start > 1 else out["actions"]
                    )
                    out.update({"best_aug_actions": gather_by_index(actions_, max_idxs)})
                    
            return out
    

In [8]:
from tensordict import TensorDict
from math import ceil
import time
import vrplib


model.eval().to(device)

results = []

for instance in instances:
    problem = vrplib.read_instance(instances[instance]["data"])

    if problem.get("node_coord", None) is None:
        print(f"Skipping {instance} as it does not have node_coord")
        continue
    coords = torch.tensor(problem['node_coord']).float()
    coords_norm = normalize_coord(coords)

    original_capacity = problem['capacity']
    demand = torch.tensor(problem['demand'][1:]).float() / original_capacity
    original_capacity = torch.tensor(original_capacity)[None]         

    # Make instance
    td_instance = TensorDict({
        "locs": coords_norm,
        "demand_linehaul": demand,
        "capacity_original": original_capacity,
    },
    batch_size=[])[None]

    td_reset = env.reset(td_instance).to(device)
    
    start = time.time()
    actions = evaluate(model, td_reset.clone(), env)["best_aug_actions"]
    inference_time = time.time() - start        

    # Obtain reward from the environment with new locs
    td_reset["locs"] = coords[None] # unnormalized
    reward = env.get_reward(td_reset, actions)
    
    # Load the optimal cost
    solution = vrplib.read_solution(instances[instance]["solution"])
    optimal_cost = solution['cost'] # note that this cost is somehow slightly lower than the one calculated from the distance matrix
    
    # Calculate the gap and print
    cost = ceil(-reward.item())
    gap = (cost - optimal_cost) / optimal_cost
    print(f'Problem: {instance:<15} Cost: {cost:<10} Optimal Cost: {optimal_cost:<10}\t Gap: {gap:.3%}')
    
    results.append({
        "instance": instance,
        "cost": cost,
        "optimal_cost": optimal_cost,
        "gap": gap,
        "inference_time": inference_time,
    })

Problem: A-n37-k6        Cost: 970        Optimal Cost: 949       	 Gap: 2.213%
Problem: A-n33-k5        Cost: 694        Optimal Cost: 661       	 Gap: 4.992%
Problem: A-n44-k6        Cost: 988        Optimal Cost: 937       	 Gap: 5.443%
Problem: A-n48-k7        Cost: 1118       Optimal Cost: 1073      	 Gap: 4.194%
Problem: A-n39-k5        Cost: 846        Optimal Cost: 822       	 Gap: 2.920%
Problem: A-n63-k10       Cost: 1331       Optimal Cost: 1314      	 Gap: 1.294%
Problem: A-n36-k5        Cost: 817        Optimal Cost: 799       	 Gap: 2.253%
Problem: A-n34-k5        Cost: 787        Optimal Cost: 778       	 Gap: 1.157%
Problem: A-n55-k9        Cost: 1120       Optimal Cost: 1073      	 Gap: 4.380%
Problem: A-n69-k9        Cost: 1187       Optimal Cost: 1159      	 Gap: 2.416%
Problem: A-n61-k9        Cost: 1071       Optimal Cost: 1034      	 Gap: 3.578%
Problem: A-n38-k5        Cost: 745        Optimal Cost: 730       	 Gap: 2.055%
Problem: A-n53-k7        Cost: 1052     

In [9]:
# Save the results with pkl
import pickle
import os 

SAVEDIR = "results/cvrplib/"
os.makedirs(SAVEDIR, exist_ok=True)

# TODO: change the filename
with open(SAVEDIR+'rf-transformer.pkl', 'wb') as f:
    pickle.dump(results, f)

### Load data

In [10]:
import pickle

# Load
with open(SAVEDIR+'rf-transformer.pkl', 'rb') as f:
    results = pickle.load(f)

# sort results by name
results = sorted(results, key=lambda x: x["instance"])

for instance in results:
    print(f'Problem: {instance["instance"]:<15} Cost: {instance["cost"]:<10} Optimal Cost: {instance["optimal_cost"]:<10}\t Gap: {instance["gap"]:.3%}')

Problem: A-n32-k5        Cost: 804        Optimal Cost: 784       	 Gap: 2.551%
Problem: A-n33-k5        Cost: 694        Optimal Cost: 661       	 Gap: 4.992%
Problem: A-n33-k6        Cost: 751        Optimal Cost: 742       	 Gap: 1.213%
Problem: A-n34-k5        Cost: 787        Optimal Cost: 778       	 Gap: 1.157%
Problem: A-n36-k5        Cost: 817        Optimal Cost: 799       	 Gap: 2.253%
Problem: A-n37-k5        Cost: 711        Optimal Cost: 669       	 Gap: 6.278%
Problem: A-n37-k6        Cost: 970        Optimal Cost: 949       	 Gap: 2.213%
Problem: A-n38-k5        Cost: 745        Optimal Cost: 730       	 Gap: 2.055%
Problem: A-n39-k5        Cost: 846        Optimal Cost: 822       	 Gap: 2.920%
Problem: A-n39-k6        Cost: 871        Optimal Cost: 831       	 Gap: 4.813%
Problem: A-n44-k6        Cost: 988        Optimal Cost: 937       	 Gap: 5.443%
Problem: A-n45-k6        Cost: 986        Optimal Cost: 944       	 Gap: 4.449%
Problem: A-n45-k7        Cost: 1156     

In [11]:
# Take the results. Measure gap depending on first letter of the instance name
gaps_names = {}
for instance in results:
    name = instance['instance'][0]
    # if name == 'X':
    #     num_locs = instance['instance'].split('-')[1][1:]
    #     if int(num_locs) < 252:
    #         name = 'X<251'
    #     elif int(num_locs) >= 252 and int(num_locs) <= 501:
    #         name = 'X251-501'
    #     else:
    #         name = 'X501-1000'
    if name not in gaps_names:
        # if X, then we divide between < 251 and >= 251
        gaps_names[name] = []
    gaps_names[name].append(instance['gap'])
        
# Calculate the average gap for each group
average_gaps = {name: sum(gaps) / len(gaps) for name, gaps in gaps_names.items()}
for name, gap in average_gaps.items():
    print(f'Group {name}: {(gap):.3%} %')

Group A: 2.825% %
Group B: 2.583% %
Group E: 2.929% %
Group F: 12.951% %
Group M: 5.078% %
Group P: 4.573% %
Group X: 8.437% %


In [12]:
# Take the results. Measure gap depending on first letter of the instance name
gaps_names = {}
for instance in results:
    name = instance['instance'][0]
    if name == 'X':
        num_locs = instance['instance'].split('-')[1][1:]
        if int(num_locs) < 252:
            name = 'X<251'
        elif int(num_locs) >= 252 and int(num_locs) <= 501:
            name = 'X251-501'
        else:
            name = 'X501-1000'
    if name not in gaps_names:
        # if X, then we divide between < 251 and >= 251
        gaps_names[name] = []
    gaps_names[name].append(instance['gap'])
        
# Calculate the average gap for each group
average_gaps = {name: sum(gaps) / len(gaps) for name, gaps in gaps_names.items()}
for name, gap in average_gaps.items():
    print(f'Group {name}: {(gap):.3%} %')

Group A: 2.825% %
Group B: 2.583% %
Group E: 2.929% %
Group F: 12.951% %
Group M: 5.078% %
Group P: 4.573% %
Group X501-1000: 12.274% %
Group X<251: 5.103% %
Group X251-501: 8.072% %
